In [ ]:
import pandas as pd
from utils import (
    df_to_excel_col_dict,
    filter_tnc_only,
    getSharesIdx,
    getSharesIdxCI95,
    link_dt,
    load_trip_assign_day,
    load_trip_raw,
    out_dir,
    prep_data_1d,
    tour_extract_wkday_dir,
    trip_dist_map,
    write_to_excel,
)
from xlsxwriter.utility import xl_rowcol_to_cell

In [ ]:
out_filepath = out_dir / "05_TNCTrips_Misc.xlsx"

In [ ]:
## Process trip records
trip = filter_tnc_only(
    load_trip_assign_day(
        tour_extract_wkday_dir / "trip-assign_day.csv", depart_hour=False
    ),
    load_trip_raw(num_travelers=False, tnc=True, transit_access_egress=False),
).to_pandas()
trip = link_dt(trip)
trip = trip_dist_map(trip)

In [ ]:
writer = pd.ExcelWriter(out_filepath, engine="xlsxwriter")
workbook = writer.book
format1 = workbook.add_format({"num_format": "#,##0.0"})

In [ ]:
for key in ["tnc_type"]:  # TODO add back in tnc_replace
    d1_dict = df_to_excel_col_dict[key]
    row = 0
    sname = d1_dict["desc"]

    title = "TNC Trips by " + d1_dict["desc"]
    tab = prep_data_1d(
        trip,
        d1_dict["desc"],
        d1_dict["col"],
        "trexpfac",
        d1_dict["vals"],
        d1_dict["labels"],
    )
    row = write_to_excel(writer, tab, sname, title, row)

    title = "Column Shares by " + d1_dict["desc"]
    row = write_to_excel(writer, getSharesIdx(tab.copy()), sname, title, row)

    tab2 = tab.copy()
    tab2.iloc[-1, -1] = tab2.iloc[-1, 0]

    title = "Column Shares 95% CI by " + d1_dict["desc"]
    row = write_to_excel(writer, getSharesIdxCI95(tab.copy()), sname, title, row)

    tab_range = xl_rowcol_to_cell(row, 1) + ":" + xl_rowcol_to_cell(row, tab.shape[1])
    _ = writer.sheets[sname].set_column(tab_range, 11, format1)

In [ ]:
writer.close()